In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
def load_csvs(folder):
    data_path = f"C:\\path\\to\\data\\{folder}"
    focal_L1 = pd.read_csv(f"{data_path}\\contrast_test_focal_L1.csv", index_col=0, header=[0, 1])
    global_L1 = pd.read_csv(f"{data_path}\\contrast_test_global.csv", index_col=0, header=[0, 1])
    textures = pd.read_csv(f"{data_path}\\contrast_test_texture.csv", index_col=0)

    def _change_expt_name(expt):
        expt = folder.replace("_nores", "") + '_' +  expt
        return expt

    focal_L1.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in focal_L1.columns])
    global_L1.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in global_L1.columns])
    textures.columns = [_change_expt_name(c) for c in textures.columns]

    return focal_L1, global_L1, textures

In [ ]:
def merge_kidney_rois(df):
    right_kidney = df["RK"]
    no_left_kidney = df["LK"].isna().any(axis=1)
    right_kidney.loc[~no_left_kidney] = (right_kidney.loc[~no_left_kidney] + df.loc[~no_left_kidney, "LK"]) / 2
    right_kidney.columns = pd.MultiIndex.from_product([["KD"], right_kidney.columns])
    df = pd.concat([df, right_kidney], axis=1)
    df.drop(["RK", "LK", "VC"], axis=1, inplace=True)
    return df

In [ ]:
focal_deep, global_deep, texture_deep = load_csvs("deep")
focal_deep = merge_kidney_rois(focal_deep)

focal_shallow, global_shallow, texture_shallow = load_csvs("shallow")
focal_shallow = merge_kidney_rois(focal_shallow)

focal_multi, global_multi, texture_multi = load_csvs("multi")
focal_multi = merge_kidney_rois(focal_multi)

focal_deep_nores, global_deep_nores, texture_deep_nores = load_csvs("deep_nores")
focal_deep_nores = merge_kidney_rois(focal_deep_nores)

focal_shallow_nores, global_shallow_nores, texture_shallow_nores = load_csvs("shallow_nores")
focal_shallow_nores = merge_kidney_rois(focal_shallow_nores)

focal_multi_nores, global_multi_nores, texture_multi_nores = load_csvs("multi_nores")
focal_multi_nores = merge_kidney_rois(focal_multi_nores)

In [ ]:
global_df = pd.concat([global_deep, global_shallow, global_multi], axis=1)
focal_df = pd.concat([focal_deep, focal_shallow, focal_multi], axis=1)
texture_df = pd.concat([texture_deep, texture_shallow, texture_multi], axis=1)
texture_df.drop(labels=["shallow_NCE", "shallow_CE", "multi_NCE", "multi_CE"], axis=1, inplace=True)
texture_df.rename({"deep_NCE": "NCE", "deep_CE": "CE"}, axis=1, inplace=True)

global_nores_df = pd.concat([global_deep_nores, global_shallow_nores, global_multi_nores], axis=1)
focal_nores_df = pd.concat([focal_deep_nores, focal_shallow_nores, focal_multi_nores], axis=1)
texture_nores_df = pd.concat([texture_deep_nores, texture_shallow_nores, texture_multi_nores], axis=1)
texture_nores_df.drop(labels=["shallow_NCE", "shallow_CE", "multi_NCE", "multi_CE"], axis=1, inplace=True)
texture_nores_df.rename({"deep_NCE": "NCE", "deep_CE": "CE"}, axis=1, inplace=True)

In [ ]:
# expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]
expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
# expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

global_df.loc[:, pd.IndexSlice[["MSE", "pSNR", "SSIM"], expts]].quantile([0.05, 0.5, 0.95])
# focal_df.loc[:, pd.IndexSlice["AO", expts]].quantile([0.05, 0.5, 0.95])
# texture_df.loc[:, ["NCE", "CE"] + expts].quantile([0.05, 0.5, 0.95])

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(global_df["MSE"])
plt.xticks(ticks=np.arange(1, global_df["MSE"].shape[1] + 1), labels=global_df["MSE"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(global_df["pSNR"])
plt.xticks(ticks=np.arange(1, global_df["pSNR"].shape[1] + 1), labels=global_df["pSNR"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(global_df["SSIM"])
plt.xticks(ticks=np.arange(1, global_df["SSIM"].shape[1] + 1), labels=global_df["SSIM"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [ ]:
print(f"Alpha = {0.05 / 6}")
# expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]
# expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

print(f"MSE resid: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR resid: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM resid: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

# expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]
# expts = ["deep_ce_vq0", "deep_ce_vq2", "deep_jo_vq512", "deep_jo_vq128"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq8", "shallow_jo_vq4", "shallow_jo_vq1024"]
expts = ["multi_ce_vq0", "multi_ce_vq32-32-128-256-1024", "multi_jo_vq128-256-512-1024-256", "multi_jo_vq32-128-512-128-64"]

print(f"MSE non-resid: {stats.kruskal(*global_nores_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR non-resid: {stats.kruskal(*global_nores_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM non-resid: {stats.kruskal(*global_nores_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(global_df["L1"])
plt.xticks(ticks=np.arange(1, global_df["L1"].shape[1] + 1), labels=global_df["L1"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(focal_df["AO"])
plt.xticks(ticks=np.arange(1, focal_df["AO"].shape[1] + 1), labels=focal_df["AO"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(focal_df["KD"])
plt.xticks(ticks=np.arange(1, focal_df["KD"].shape[1] + 1), labels=focal_df["KD"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [ ]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]

print("Residual")
print(f"Global L1: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")

print(f"Focal AO deep vs shallow: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'deep_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'shallow_ce_vq0']])}")
print(f"Focal AO deep vs multi: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'deep_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']])}")
print(f"Focal AO shallow vs multi: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'shallow_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']])}")

print("Non-residual")
print(f"Global L1: {stats.kruskal(*global_nores_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_nores_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_nores_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_nores_df.loc[:, expts].values.T.tolist())}")

print(f"Texture deep vs shallow: {stats.mannwhitneyu(texture_nores_df.loc[:, 'deep_ce_vq0'], texture_nores_df.loc[:, 'shallow_ce_vq0'])}")
print(f"Texture deep vs multi: {stats.mannwhitneyu(texture_nores_df.loc[:, 'deep_ce_vq0'], texture_nores_df.loc[:, 'multi_ce_vq0'])}")
print(f"Texture shallow vs multi: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_ce_vq0'], texture_nores_df.loc[:, 'multi_ce_vq0'])}")

In [ ]:
print(f"Alpha = {0.05 / 12}")
# expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

print("Residual")
print(f"Global L1: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")

print("Multi")
print(f"Focal AO base vs vq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']])}")
print(f"Focal AO base vs jvq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"Focal AO base vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")
print(f"Focal AO vq-ce vs jvq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"Focal AO vq-ce vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")
print(f"Focal AO jvq-ce vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")

print("Texture")
print(f"Texture base vs vq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture vq-ce vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture vq-ce vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture jvq-ce vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")

# expts = ["deep_ce_vq0", "deep_ce_vq2", "deep_jo_vq512", "deep_jo_vq128"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq8", "shallow_jo_vq4", "shallow_jo_vq1024"]
expts = ["multi_ce_vq0", "multi_ce_vq32-32-128-256-1024", "multi_jo_vq128-256-512-1024-256", "multi_jo_vq32-128-512-128-64"]

print("Non-residual")
print(f"Global L1: {stats.kruskal(*global_nores_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_nores_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_nores_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_nores_df.loc[:, expts].values.T.tolist())}")

In [ ]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]

print()

for expt in expts:
    print(f"MSE {expt} resid vs non-resid: {stats.mannwhitneyu(global_df.loc[:, pd.IndexSlice['MSE', expt]], global_nores_df.loc[:, pd.IndexSlice['MSE', expt]])}")
    print(f"pSNR {expt} resid vs non-resid: {stats.mannwhitneyu(global_df.loc[:, pd.IndexSlice['pSNR', expt]], global_nores_df.loc[:, pd.IndexSlice['pSNR', expt]])}")
    print(f"SSIM {expt} resid vs non-resid: {stats.mannwhitneyu(global_df.loc[:, pd.IndexSlice['SSIM', expt]], global_nores_df.loc[:, pd.IndexSlice['SSIM', expt]])}")

print()

for expt in expts:
    print(f"Focal AO {expt} resid vs non-resid: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', expt]], focal_nores_df.loc[:, pd.IndexSlice['AO', expt]])}")
    print(f"Focal KD {expt} resid vs non-resid: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['KD', expt]], focal_nores_df.loc[:, pd.IndexSlice['KD', expt]])}")
    print(f"Texture {expt} resid vs non-resid: {stats.mannwhitneyu(texture_df.loc[:, expt], texture_nores_df.loc[:, expt])}")